In [1]:
import numpy as np
from scipy import spatial,stats
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
window_items = [1, 257, 4, 5, 8, 9, 10, 234, 12, 205, 235, 207, 239, 241, 211, 213, 216, 250, 219, 220, 255] 
target_items = [2, 3, 7, 11, 13, 17, 215, 236]

In [230]:
import os,sys
my_path = os.getcwd()
print (my_path)
path = os.path.join(my_path, "../data/ratings.txt")
print (path)

/media/thepurpleowl/FCC8B519C8B4D35E/Academics_research/research/BKPatra/Attacks_RS/AttackProfileDetectionRS/filmtrust/User-Based-Recommendation
/media/thepurpleowl/FCC8B519C8B4D35E/Academics_research/research/BKPatra/Attacks_RS/AttackProfileDetectionRS/filmtrust/User-Based-Recommendation/../data/ratings.txt


In [256]:
train = pd.read_csv(path,sep=' ',header=None,names=['userId','itemId','rating'])

In [257]:
train.describe()

,userId,itemId,rating
count,35497.000000,35497.000000,35497.000000
mean,744.247824,265.524946,3.002803
std,433.199457,346.854769,0.918692
min,1.000000,1.000000,0.500000
25%,356.000000,12.000000,2.500000
50%,735.000000,216.000000,3.000000
75%,1130.000000,251.000000,4.000000
max,1508.000000,2071.000000,4.000000


### Actual Average Rating of each item 

In [258]:
calculated_Rating = {}
items = train['itemId'].unique()

#rating
R = train.groupby(['itemId'])['rating'].mean()
for i in items:
#     print (i)
    calculated_Rating[i] = R[i]
    
# print (calculated_Rating)

#### with simple train-test(previously used split) 
test = pd.DataFrame()  
for t_i in target_items:  
    temp_df = train[train['itemId'] == t_i]  
    train =  train.drop(train[train['itemId'] == t_i].index)  
    test = pd.concat([temp_df, test]).drop_duplicates(keep=False)  

### Checking by only taking users who have rated more than 100 items :Conclusion : strong attack 

In [259]:
temp_test = pd.DataFrame()  
for t_i in target_items:  
    temp_df = train[train['itemId'] == t_i]  
    train =  train.drop(train[train['itemId'] == t_i].index)  
    temp_test = pd.concat([temp_df, temp_test]).drop_duplicates(keep=False)  
    
# print (temp_test.shape)

uu = []  
test = pd.DataFrame()  

for index1,i in enumerate(temp_test.itertuples(),0):  
    
    usr=int(i.userId)  
    mov=int(i.itemId)  
#     print (index1)    
    list_of_movies_current_user_has_rated=train[train['userId']==usr]  
    if(list_of_movies_current_user_has_rated.shape[0]<100):  
        pass                  #test = test.drop(test.index[index1])  
    else:  
        k = pd.Series([i.userId,i.itemId,i.rating])  
        test = test.append(k, ignore_index=True)  
        uu.append(usr)  
print (len(uu))  

print (test.shape)  
test.columns = ['userId','itemId','rating']  
print (test.head(5))  

84
(84, 3)
   userId  itemId  rating
0   161.0   236.0     3.0
1   188.0   236.0     4.0
2   199.0   236.0     2.5
3   272.0   236.0     3.0
4   323.0   236.0     3.0


### testing code of divising test/target data
table = [['Heading1', 'Heading2'], [1 , 1],[1 , 2],[1 , 3],[1 , 4], [1,5],[2 , 1],[2 , 2],[2 , 3],[2 , 4],[3, 1],[3, 2],[3, 3],[3, 4]]  
headers = table.pop(0)  
df = pd.DataFrame(table, columns=headers)

In [260]:
dfs = {n: g for n, g in test.groupby(test.groupby('itemId').cumcount() % 10)} 
test = pd.concat([dfs[0],dfs[1]], ignore_index=False)
train = pd.concat([train,dfs[2],dfs[3],dfs[4],dfs[5]], ignore_index=False)
del_data = pd.concat([dfs[6],dfs[7],dfs[8],dfs[9]], ignore_index=False)

In [261]:
train.shape[0]

28798

In [262]:
test.shape[0] 

25

In [263]:
test.tail(5)

,userId,itemId,rating
51,188.0,7.0,1.5
61,1333.0,7.0,1.0
63,161.0,3.0,3.5
73,1333.0,3.0,4.0
75,272.0,2.0,3.5


In [264]:
train.tail(5)

,userId,itemId,rating
26,323.0,17.0,2.5
39,323.0,13.0,2.5
55,670.0,7.0,4.0
67,670.0,3.0,3.5
79,1039.0,2.0,3.5


### Finding error for target items in actual data

In [265]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{17: [0, 0], 2: [0, 0], 3: [0, 0], 7: [0, 0], 11: [0, 0], 236: [0, 0], 13: [0, 0], 215: [0, 0]}


In [266]:
test.head(5)

,userId,itemId,rating
0,161.0,236.0,3.0
10,1249.0,236.0,3.5
12,79.0,215.0,1.0
21,3.0,17.0,3.0
31,1199.0,17.0,3.0


In [267]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['userId']==usr]
#     print (list_of_movies_current_user_has_rated)
#     print (all_ratings_for_the_active_item.shape)
#     print (list_of_movies_current_user_has_rated.shape)
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(all_ratings_for_the_active_item.itertuples(),1): 
        c1=[]
        c2=[]
        c_user=int(u.userId)
        list_curuser=train[train['userId']==c_user]
#         print (list_curuser)
        xt=pd.merge( list_curuser,list_of_movies_current_user_has_rated,how='inner',on=['itemId'])
#         print (xt)
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
#         print (c1_list,'          ',c2_list)
        sim=0
        if len(c1_list)!=0:
            sim = (1-spatial.distance.cosine(c1_list,c2_list)) #stats.pearsonr(c1_list,c2_list)[0]
#             print (sim)
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

In [268]:
index

25

In [269]:
print (t2-t1)

0.37560105323791504


In [270]:
target_item_error_in_actual_data

{2: [0.21321060953177495, 2],
 3: [2.5923876400820012, 4],
 7: [0.49892328859986845, 4],
 11: [1.9285584549686083, 2],
 13: [0.33776229319588236, 3],
 17: [1.5097665742774558, 4],
 215: [1.4843402841025242, 2],
 236: [0.53429288546655274, 4]}

### MAE error

In [271]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

17     : 0.377441643569
2     : 0.106605304766
3     : 0.648096910021
7     : 0.12473082215
11     : 0.964279227484
236     : 0.133573221367
13     : 0.112587431065
215     : 0.742170142051


## Nuke Attack Strength Check

### Injecting Attack Profiles

In [272]:
nuke_rating = 0.5

In [273]:
attack_ids = list(range(1600,1700))

for attacker in attack_ids:
    for item in window_items:
        a = calculated_Rating[item]
        attack_rating = int(a) if (a-int(a)<0.5) else int(a)+1
        train = train.append(pd.Series([attacker,item,attack_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

for attacker in attack_ids:
    for item in target_items:
        train = train.append(pd.Series([attacker,item,nuke_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

In [274]:
train.shape[0]

31698

In [275]:
train.tail(5)

,userId,itemId,rating
31693,1699.0,11.0,0.5
31694,1699.0,13.0,0.5
31695,1699.0,17.0,0.5
31696,1699.0,215.0,0.5
31697,1699.0,236.0,0.5


## Finding error for target items in attack profile injected data

In [276]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{17: [0, 0], 2: [0, 0], 3: [0, 0], 7: [0, 0], 11: [0, 0], 236: [0, 0], 13: [0, 0], 215: [0, 0]}


In [277]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['userId']==usr]
#     print (all_ratings_for_the_active_item.shape)
#     print (list_of_movies_current_user_has_rated.shape)
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(all_ratings_for_the_active_item.itertuples(),1): 
        c1=[]
        c2=[]
        c_user=int(u.userId)
#         c_item=int(u.itemId)
        list_curuser=train[train['userId']==c_user]
#         list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( list_of_movies_current_user_has_rated,list_curuser,how='inner',on=['itemId'])
#         print (list_curuser)
#         print (list_of_movies_current_user_has_rated)
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim= (1-spatial.distance.cosine(c1_list,c2_list))#stats.pearsonr(c1_list,c2_list)[0]
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

In [278]:
print (t2-t1)

7.41774582862854


In [279]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

17     : 2.46627414527
2     : 2.23750774621
3     : 2.4008416197
7     : 2.38498907462
11     : 2.57988762558
236     : 2.37773651859
13     : 2.56146539277
215     : 2.66590822011
